In [1]:
import pandas as pd
import sqlalchemy
from binance import Client
from binance import BinanceSocketManager
import asyncio

In [2]:
with open('access.txt') as f:
    acc = f.readlines()
api = acc[0].strip()
key = acc[1].strip()

In [3]:
client = Client(api, key)

In [4]:
def create_df(message):
    df = pd.DataFrame([message])
    df = df.loc[:,['s','E','p','q']]
    df.E = pd.to_datetime(df.E, unit='ms') + pd.Timedelta('03:00:00')
    df[['p','q']] =  df[['p','q']].astype('float')
    df.columns = ['Symbol','Time','Price','Quantity']
    return df

In [5]:
# create stream and drop table
engine = sqlalchemy.create_engine('sqlite:///DogeTrade.db')
conn = engine.connect()
conn.execute(f'DROP TABLE IF EXISTS doge_test_1')

In [6]:
# start stream
async def start_stream():
    bsm = BinanceSocketManager(client)
    socket = bsm.trade_socket('DOGEUSDT')
    while True:
        await socket.__aenter__()
        msg = await socket.recv()
        df = create_df(msg)
        df.to_sql('doge_test_1', engine, if_exists='append', index=False)

In [ ]:
#await start_stream()